<a href="https://colab.research.google.com/github/tatendatobaiwa/cnn/blob/main/cnnmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 2: Create Directories in Drive

Step 3: Install Dependenices

In [21]:
!pip install tensorflow keras numpy pandas matplotlib scikit-learn opencv-python seaborn

Step 4: Use a script to Load and Preprocess Data

In [16]:
import os
from pathlib import Path
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_images_from_directory(directory: Path, target_size: tuple = (256, 256)) -> tuple:
    """
    Load images from class subdirectories with enhanced error handling
    """
    images = []
    labels = []
    class_names = ['benign', 'malignant', 'normal']

    for label, class_name in enumerate(class_names):
        class_dir = directory / class_name
        if not class_dir.exists():
            raise FileNotFoundError(f"Class directory {class_dir} not found")

        for img_path in class_dir.glob('*'):
            if img_path.suffix.lower() in ('.png', '.jpg', '.jpeg'):
                try:
                    img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
                    if img is None:
                        continue
                    img = cv2.resize(img, target_size)
                    images.append(img)
                    labels.append(label)
                except Exception as e:
                    print(f"Error processing {img_path}: {str(e)}")

    return np.array(images), np.array(labels)

def preprocess_data(data_dir: str, test_dir: str, target_size: tuple = (256, 256),
                   validation_split: float = 0.2, batch_size: int = 32) -> tuple:
    """
    Complete data pipeline with proper train-val-test split
    """
    # Convert to Path objects
    data_path = Path(data_dir)
    test_path = Path(test_dir)

    # Load and split data
    X, y = load_images_from_directory(data_path / 'train')
    X = X[..., np.newaxis] / 255.0  # Normalize and add channel dimension

    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=validation_split, stratify=y, random_state=42
    )

    # Data augmentation configuration
    train_datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    val_datagen = ImageDataGenerator()

    # Create generators
    train_generator = train_datagen.flow(
        X_train, y_train, batch_size=batch_size, shuffle=True
    )

    val_generator = val_datagen.flow(
        X_val, y_val, batch_size=batch_size, shuffle=False
    )

    # Load test data
    test_images = []
    test_files = []
    for img_path in test_path.glob('*'):
        if img_path.suffix.lower() in ('.png', '.jpg', '.jpeg'):
            img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, target_size)
            test_images.append(img)
            test_files.append(img_path.name)

    test_images = np.array(test_images)[..., np.newaxis] / 255.0

    return train_generator, val_generator, test_images, test_files, X_val, y_val

Train images: 28
Validation images: 7
Test images: 197


Step 5: Model Development

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def create_cnn_model(input_shape: tuple = (256, 256, 1)) -> Sequential:
    """
    Optimized CNN architecture with proper regularization
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.3),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.4),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.5),

        Flatten(),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.6),
        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def train_model(model: Sequential, train_gen, val_gen, epochs: int = 50,
               model_path: str = 'models/best_model.h5') -> dict:
    """
    Training process with checkpointing and early stopping
    """
    callbacks = [
        EarlyStopping(patience=8, restore_best_weights=True),
        ModelCheckpoint(model_path, save_best_only=True)
    ]

    history = model.fit(
        train_gen,
        validation_data=val_gen,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1
    )
    return history.history

Step 5: Model Evaluation and Optimization

In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

def evaluate_model(model, X_val, y_val, class_names: list = ['benign', 'malignant', 'normal']):
    """
    Comprehensive model evaluation with visualization
    """
    # Generate predictions
    y_pred = model.predict(X_val)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Classification report
    print("Classification Report:")
    print(classification_report(y_val, y_pred_classes, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(y_val, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

    # Calculate accuracy per class
    class_acc = {}
    for i, class_name in enumerate(class_names):
        mask = y_val == i
        class_acc[class_name] = np.mean(y_pred_classes[mask] == y_val[mask])

    print("\nClass-wise Accuracy:")
    for class_name, acc in class_acc.items():
        print(f"{class_name}: {acc:.2%}")

Step 6: Explainable AI (XAI) Integration

In [27]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

def grad_cam(model, img_array, layer_name: str = 'conv2d_2', class_index: int = None):
    """
    Enhanced Grad-CAM implementation with proper tensor handling
    """
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if class_index is None:
            class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap).numpy()

    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    # Resize and convert to RGB
    heatmap = cv2.resize(heatmap, (img_array.shape[2], img_array.shape[1]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # Superimpose with original image
    img = np.uint8(255 * img_array[0].squeeze())
    superimposed_img = cv2.addWeighted(cv2.cvtColor(img, cv2.COLOR_GRAY2BGR), 0.6, heatmap, 0.4, 0)

    return superimposed_img

def visualize_explanations(model, X_val, y_val, num_samples: int = 3):
    """
    Visualize predictions with Grad-CAM explanations
    """
    class_names = ['benign', 'malignant', 'normal']
    indices = np.random.choice(len(X_val), num_samples, replace=False)

    plt.figure(figsize=(15, 5 * num_samples))
    for i, idx in enumerate(indices, 1):
        img = X_val[idx]
        true_label = class_names[y_val[idx]]

        # Get prediction
        pred = model.predict(np.expand_dims(img, axis=0))
        pred_class = class_names[np.argmax(pred)]

        # Generate explanation
        explanation = grad_cam(model, np.expand_dims(img, axis=0))

        # Plot results
        plt.subplot(num_samples, 2, 2*i-1)
        plt.imshow(img.squeeze(), cmap='gray')
        plt.title(f"True: {true_label}\nPred: {pred_class}")
        plt.axis('off')

        plt.subplot(num_samples, 2, 2*i)
        plt.imshow(explanation)
        plt.title("Grad-CAM Explanation")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

Step 7: Train Model

In [29]:
from pathlib import Path
from src.data import preprocess_data
from src.model import create_cnn_model, train_model
from src.evaluate import evaluate_model
from src.xai import visualize_explanations

# Configuration
DATA_DIR = Path('data/raw')
TEST_DIR = DATA_DIR / 'test'
MODEL_PATH = 'models/best_model.h5'
TARGET_SIZE = (256, 256)
EPOCHS = 50
BATCH_SIZE = 32

def main():
    # Data preparation
    train_gen, val_gen, test_images, test_files, X_val, y_val = preprocess_data(
        DATA_DIR, TEST_DIR, target_size=TARGET_SIZE, batch_size=BATCH_SIZE
    )

    # Model training
    model = create_cnn_model(input_shape=(*TARGET_SIZE, 1))
    history = train_model(model, train_gen, val_gen, epochs=EPOCHS, model_path=MODEL_PATH)

    # Evaluation
    print("\nModel Evaluation:")
    evaluate_model(model, X_val, y_val)

    # Explainability
    print("\nGenerating Explanations...")
    visualize_explanations(model, X_val, y_val, num_samples=3)

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'src'